In [1]:
import os   

In [2]:
%pwd

'd:\\DS\\projects\\text summaris\\Text-Summarizer-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\DS\\projects\\text summaris\\Text-Summarizer-Project'

In [76]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file : Path
    unzip_dir : Path
    

In [77]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml , create_directories


In [78]:
read_yaml(CONFIG_FILE_PATH).data_ingestion.local_data_file

[2023-06-26 17:01:33,973]: INFO:common:yaml file:Config\config.yaml loaded successfully]


'artifacts/data_ingestion/data.zip'

In [79]:
class ConfiguarationManager:
    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath= PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url= config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir= config.unzip_dir

        )
        return data_ingestion_config 
    

In [80]:
#components 
import os 
import urllib.request as request 
import zipfile
from textSummarizer.logging import logger 
from textSummarizer.utils.common import get_size

  

In [81]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig) -> None:
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            print(self.config.source_url)
            print(self.config.local_data_file)
            filename , headers = request.urlretrieve(
                url = self.config.source_url,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"file already exist of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path:str
        Extracts the zip file into the data directory
        Function returns None
        """
                
        unzip_path = self.config.unzip_dir
 
        os.makedirs(unzip_path,exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:

            zip_ref.extractall(unzip_path)



In [83]:
#create pipeline 
try: 
    config = ConfiguarationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e 

[2023-06-26 17:03:00,095]: INFO:common:yaml file:Config\config.yaml loaded successfully]
[2023-06-26 17:03:00,100]: INFO:common:yaml file:params.yaml loaded successfully]
[2023-06-26 17:03:00,102]: INFO:common:created directory at :artifacts ]
[2023-06-26 17:03:00,105]: INFO:common:created directory at :artifacts/data_ingestion ]
https://github.com/entbappy/Branching-tutorial/raw/master/samsumdata.zip
artifacts/data_ingestion/data.zip
[2023-06-26 17:03:04,753]: INFO:1239740481:artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23627009
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "2e7409b328d118a1d37018be788babf8bf9640386387da766ac100e96efa3b93"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B634:327D:299673:3283E6:649971D0
Accept-Ranges: 

In [39]:
config.source_url

AttributeError: 'ConfiguarationManager' object has no attribute 'source_url'

In [75]:
#components 
import os 
import urllib.request as request 
import zipfile

filename , headers = request.urlretrieve(
                url = 'https://github.com/entbappy/Branching-tutorial/raw/master/samsumdata.zip',
                filename= r'D:\DS\projects\text summaris\Text-Summarizer-Project\artifacts\data_ingestion\data.zip'
            )


target_dir = r'D:\DS\projects\text summaris\Text-Summarizer-Project\artifacts\data_ingestion'
zip_path = filename
os.makedirs(target_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(target_dir)

EOFError: 